In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import math
import datetime as dt
import tqdm

In [2]:
df = pd.read_csv("D:/NLP/Volatility/Data/SP500/all_stocks_5yr.csv")
df.head()
# df.tail()

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [3]:
# Get the list of company names
list_comp = df['Name'].unique()
num_comp = list_comp.shape[0]
print (num_comp, type(list_comp))
list_comp

505 <class 'numpy.ndarray'>


array(['AAL', 'AAPL', 'AAP', 'ABBV', 'ABC', 'ABT', 'ACN', 'ADBE', 'ADI',
       'ADM', 'ADP', 'ADSK', 'ADS', 'AEE', 'AEP', 'AES', 'AET', 'AFL',
       'AGN', 'AIG', 'AIV', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK',
       'ALLE', 'ALL', 'ALXN', 'AMAT', 'AMD', 'AME', 'AMGN', 'AMG', 'AMP',
       'AMT', 'AMZN', 'ANDV', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'APC',
       'APD', 'APH', 'APTV', 'ARE', 'ARNC', 'ATVI', 'AVB', 'AVGO', 'AVY',
       'AWK', 'AXP', 'AYI', 'AZO', 'A', 'BAC', 'BAX', 'BA', 'BBT', 'BBY',
       'BDX', 'BEN', 'BF.B', 'BHF', 'BHGE', 'BIIB', 'BK', 'BLK', 'BLL',
       'BMY', 'BRK.B', 'BSX', 'BWA', 'BXP', 'CAG', 'CAH', 'CAT', 'CA',
       'CBG', 'CBOE', 'CBS', 'CB', 'CCI', 'CCL', 'CDNS', 'CELG', 'CERN',
       'CFG', 'CF', 'CHD', 'CHK', 'CHRW', 'CHTR', 'CINF', 'CI', 'CLX',
       'CL', 'CMA', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP',
       'COF', 'COG', 'COL', 'COO', 'COP', 'COST', 'COTY', 'CPB', 'CRM',
       'CSCO', 'CSRA', 'CSX', 'CTAS', 'CTL', 'CTSH', 'CTXS

In [4]:
mask = df['Name'] == 'AAL'
df[mask]
df.loc[df['Name'] == "AAL"]
df['date'] = pd.to_datetime(df.date, format='%Y-%m-%d')
# Extract all data from year 2017
mask = df['date'].dt.year == int(2017)
include = df[mask]
exclude = df[~mask]
df_17 = df[mask]
df_17

,date,open,high,low,close,volume,Name
982,2017-01-03,47.28,47.340,46.135,46.30,6737752,AAL
983,2017-01-04,46.63,47.435,46.350,46.70,5859604,AAL
984,2017-01-05,46.52,46.930,45.610,45.89,6825316,AAL
985,2017-01-06,45.85,46.720,45.470,46.21,7260197,AAL
986,2017-01-09,46.01,47.340,45.780,47.08,4739142,AAL
...,...,...,...,...,...,...,...
619009,2017-12-22,72.30,72.370,71.790,71.99,1345683,ZTS
619010,2017-12-26,72.40,72.550,71.900,72.34,792134,ZTS
619011,2017-12-27,72.59,72.690,72.250,72.45,1159771,ZTS
619012,2017-12-28,72.49,72.600,72.140,72.39,710499,ZTS


In [5]:
start_date = df_17.iloc[0].date
end_date = df_17.iloc[-1].date
print (start_date, end_date)


2017-01-03 00:00:00 2017-12-29 00:00:00


In [6]:
lastdayfrom = pd.to_datetime('2017-01-03')
lastdayfrom

Timestamp('2017-01-03 00:00:00')

In [7]:
tau = 3 # This is a variable. Values to be used are 3, 7, 15, and 30

# Create a mask for a specific stock
mask_stock = df['Name'] == 'AAL'
df_stock = df[mask_stock]
# Go back 50 calendar days to include more than 30 trading sessions
df_adj = df_stock[df_stock["date"] >= (pd.to_datetime('2017-01-03') - pd.Timedelta(days=50))]
df_adj.head(10)

,date,open,high,low,close,volume,Name
949,2016-11-14,43.01,43.610,42.98,43.40,7773627,AAL
950,2016-11-15,45.00,45.280,43.59,44.76,13166625,AAL
951,2016-11-16,44.66,44.795,43.93,44.45,6166966,AAL
952,2016-11-17,44.39,45.850,44.05,45.83,8558682,AAL
953,2016-11-18,46.15,46.490,45.62,46.26,7795053,AAL
954,2016-11-21,46.46,46.750,45.95,46.08,6651810,AAL
955,2016-11-22,46.03,46.950,45.95,46.08,5781259,AAL
956,2016-11-23,46.05,46.390,45.70,46.34,5160176,AAL
957,2016-11-25,46.34,46.860,46.30,46.82,2426632,AAL
958,2016-11-28,46.64,46.885,45.66,46.50,7369639,AAL


In [8]:
# Reindex for better data access
df_adj.index = np.arange(1, len(df_adj) + 1)
df_adj.head()

,date,open,high,low,close,volume,Name
1,2016-11-14,43.01,43.610,42.98,43.40,7773627,AAL
2,2016-11-15,45.00,45.280,43.59,44.76,13166625,AAL
3,2016-11-16,44.66,44.795,43.93,44.45,6166966,AAL
4,2016-11-17,44.39,45.850,44.05,45.83,8558682,AAL
5,2016-11-18,46.15,46.490,45.62,46.26,7795053,AAL


In [9]:
# Calculate starting index of starting and end dates
start_idx = int(df_adj.index[df_adj['date'] == start_date][0])
end_idx = int(df_adj.index[df_adj['date'] == end_date][0])
print (start_idx, end_idx)


34 284


In [10]:
Pt = df_adj.loc[start_idx]['close'] 
Pt_minus1 = df_adj.loc[(start_idx-1)]['close'] 
print(Pt , Pt_minus1)

46.3 46.69


In [11]:
def get_volatility(df_adjusted, time, index):
    mean_return_price = get_mean_return_price(df_adjusted, index, time)
    sqr_sum = 0
    for i in range(time + 1):
        Pt = df_adjusted.loc[index]['close']
        Pt_minus1 = df_adjusted.loc[(index - 1)]['close'] 
        return_price = get_return_price(Pt, Pt_minus1)
        temp_sqr_sum = np.square(return_price - mean_return_price)
        sqr_sum = sqr_sum + temp_sqr_sum
    volatility = np.log(np.sqrt(sqr_sum/time))
    return volatility
    

In [12]:
def get_return_price(close_price, close_price_minus1):
    return_price = (close_price/close_price_minus1) - 1
    return return_price

In [13]:
def get_mean_return_price(df, idx, time):
    mean_return_price = np.mean(df['close'].loc[range(idx,idx - time - 1,-1)])
    return mean_return_price
    

In [14]:
# Retrieve stock specific DataFrame based on stock name
def get_stock_specific_df(df, stock_symbol):
    df_copy = df.copy()
    mask_stock = df_copy['Name'] == stock_symbol
    df_stock = df_copy[mask_stock]
    # Go back 50 calendar days to include more than 30 trading sessions
    df_stock = df_stock[df_stock["date"] >= (pd.to_datetime('2017-01-03') - pd.Timedelta(days=50))]
    df_stock.index = np.arange(1, len(df_stock) + 1)
    return df_stock
    
    
    

In [15]:
# Test Block to verify indices
np.sum(df_adj.loc[range(start_idx, start_idx - tau -1, -1)]['close'])

187.91000000000003

In [16]:
# Drop company names whose full year data is not available
# Indices to drop: 47, 
list_comp = df['Name'].unique()
drop_index = [47, 67, 68, 150, 151,226]
list_comp_new = np.delete(list_comp, drop_index)

# print(list_comp) 
print(np.setdiff1d(list_comp, list_comp_new))
print(list_comp[226])

['APTV' 'BHF' 'BHGE' 'DWDP' 'DXC' 'HLT']
HLT


In [17]:
# Create empty dataframe to store volatility values
df_volatility = pd.DataFrame(columns=['stock', 'date', 'close', 'vol3', 'vol7', 'vol15', 'vol30'])
print(list_comp_new)

['AAL' 'AAPL' 'AAP' 'ABBV' 'ABC' 'ABT' 'ACN' 'ADBE' 'ADI' 'ADM' 'ADP'
 'ADSK' 'ADS' 'AEE' 'AEP' 'AES' 'AET' 'AFL' 'AGN' 'AIG' 'AIV' 'AIZ' 'AJG'
 'AKAM' 'ALB' 'ALGN' 'ALK' 'ALLE' 'ALL' 'ALXN' 'AMAT' 'AMD' 'AME' 'AMGN'
 'AMG' 'AMP' 'AMT' 'AMZN' 'ANDV' 'ANSS' 'ANTM' 'AON' 'AOS' 'APA' 'APC'
 'APD' 'APH' 'ARE' 'ARNC' 'ATVI' 'AVB' 'AVGO' 'AVY' 'AWK' 'AXP' 'AYI'
 'AZO' 'A' 'BAC' 'BAX' 'BA' 'BBT' 'BBY' 'BDX' 'BEN' 'BF.B' 'BIIB' 'BK'
 'BLK' 'BLL' 'BMY' 'BRK.B' 'BSX' 'BWA' 'BXP' 'CAG' 'CAH' 'CAT' 'CA' 'CBG'
 'CBOE' 'CBS' 'CB' 'CCI' 'CCL' 'CDNS' 'CELG' 'CERN' 'CFG' 'CF' 'CHD' 'CHK'
 'CHRW' 'CHTR' 'CINF' 'CI' 'CLX' 'CL' 'CMA' 'CMCSA' 'CME' 'CMG' 'CMI'
 'CMS' 'CNC' 'CNP' 'COF' 'COG' 'COL' 'COO' 'COP' 'COST' 'COTY' 'CPB' 'CRM'
 'CSCO' 'CSRA' 'CSX' 'CTAS' 'CTL' 'CTSH' 'CTXS' 'CVS' 'CVX' 'CXO' 'C'
 'DAL' 'DE' 'DFS' 'DGX' 'DG' 'DHI' 'DHR' 'DISCA' 'DISCK' 'DISH' 'DIS'
 'DLR' 'DLTR' 'DOV' 'DPS' 'DRE' 'DRI' 'DTE' 'DUK' 'DVA' 'DVN' 'D' 'EA'
 'EBAY' 'ECL' 'ED' 'EFX' 'EIX' 'EL' 'EMN' 'EMR' 'EOG' 'EQIX' 'EQR'

In [18]:
# Main function to calculate all period volatility for all stocks
for idx, name in enumerate(list_comp_new):
    print (" processing ", idx+1, " of: ", len(list_comp_new),  name)
    df_stock = get_stock_specific_df(df, name)
    for i in range (start_idx, end_idx+1):
        date = df_stock.loc[i]['date']
        close = df_stock.loc[i]['close']
        volatility_3 = get_volatility(df_stock, 3, i)
        volatility_7 = get_volatility(df_stock, 7, i)
        volatility_15 = get_volatility(df_stock, 15, i)
        volatility_30 = get_volatility(df_stock, 30, i)
        df_volatility = pd.concat([pd.DataFrame([[name,date,close,volatility_3,volatility_7,volatility_15,volatility_30]],
                               columns=df_volatility.columns), df_volatility], ignore_index=False)


 processing  1  of:  499 AAL
 processing  2  of:  499 AAPL
 processing  3  of:  499 AAP
 processing  4  of:  499 ABBV
 processing  5  of:  499 ABC
 processing  6  of:  499 ABT
 processing  7  of:  499 ACN
 processing  8  of:  499 ADBE
 processing  9  of:  499 ADI
 processing  10  of:  499 ADM
 processing  11  of:  499 ADP
 processing  12  of:  499 ADSK
 processing  13  of:  499 ADS
 processing  14  of:  499 AEE
 processing  15  of:  499 AEP
 processing  16  of:  499 AES
 processing  17  of:  499 AET
 processing  18  of:  499 AFL
 processing  19  of:  499 AGN
 processing  20  of:  499 AIG
 processing  21  of:  499 AIV
 processing  22  of:  499 AIZ
 processing  23  of:  499 AJG
 processing  24  of:  499 AKAM
 processing  25  of:  499 ALB
 processing  26  of:  499 ALGN
 processing  27  of:  499 ALK
 processing  28  of:  499 ALLE
 processing  29  of:  499 ALL
 processing  30  of:  499 ALXN
 processing  31  of:  499 AMAT
 processing  32  of:  499 AMD
 processing  33  of:  499 AME
 processin

 processing  268  of:  499 KO
 processing  269  of:  499 KR
 processing  270  of:  499 KSS
 processing  271  of:  499 KSU
 processing  272  of:  499 K
 processing  273  of:  499 LB
 processing  274  of:  499 LEG
 processing  275  of:  499 LEN
 processing  276  of:  499 LH
 processing  277  of:  499 LKQ
 processing  278  of:  499 LLL
 processing  279  of:  499 LLY
 processing  280  of:  499 LMT
 processing  281  of:  499 LNC
 processing  282  of:  499 LNT
 processing  283  of:  499 LOW
 processing  284  of:  499 LRCX
 processing  285  of:  499 LUK
 processing  286  of:  499 LUV
 processing  287  of:  499 LYB
 processing  288  of:  499 L
 processing  289  of:  499 MAA
 processing  290  of:  499 MAC
 processing  291  of:  499 MAR
 processing  292  of:  499 MAS
 processing  293  of:  499 MAT
 processing  294  of:  499 MA
 processing  295  of:  499 MCD
 processing  296  of:  499 MCHP
 processing  297  of:  499 MCK
 processing  298  of:  499 MCO
 processing  299  of:  499 MDLZ
 processing  3

In [19]:
df_volatility

,stock,date,close,vol3,vol7,vol15,vol30
0,ZTS,2017-12-29,72.04,4.424801,4.348693,4.311820,4.288958
0,ZTS,2017-12-28,72.39,4.424573,4.349657,4.310837,4.287938
0,ZTS,2017-12-27,72.45,4.424160,4.348882,4.309470,4.286324
0,ZTS,2017-12-26,72.34,4.425013,4.347644,4.308182,4.284778
0,ZTS,2017-12-22,71.99,4.426724,4.347835,4.308270,4.283528
...,...,...,...,...,...,...,...
0,AAL,2017-01-09,47.08,3.982243,3.910615,3.896242,3.874654
0,AAL,2017-01-06,46.21,3.978292,3.914955,3.897592,3.874223
0,AAL,2017-01-05,45.89,3.981407,3.921501,3.900233,3.874648
0,AAL,2017-01-04,46.70,3.988150,3.928339,3.901909,3.874351


In [20]:
# Reverse the df
df_volatility = df_volatility[::-1]
# Reindex for better data access
df_volatility.index = np.arange(1, len(df_volatility) + 1)
df_volatility

,stock,date,close,vol3,vol7,vol15,vol30
1,AAL,2017-01-03,46.30,3.993688,3.935951,3.906089,3.874118
2,AAL,2017-01-04,46.70,3.988150,3.928339,3.901909,3.874351
3,AAL,2017-01-05,45.89,3.981407,3.921501,3.900233,3.874648
4,AAL,2017-01-06,46.21,3.978292,3.914955,3.897592,3.874223
5,AAL,2017-01-09,47.08,3.982243,3.910615,3.896242,3.874654
...,...,...,...,...,...,...,...
125245,ZTS,2017-12-22,71.99,4.426724,4.347835,4.308270,4.283528
125246,ZTS,2017-12-26,72.34,4.425013,4.347644,4.308182,4.284778
125247,ZTS,2017-12-27,72.45,4.424160,4.348882,4.309470,4.286324
125248,ZTS,2017-12-28,72.39,4.424573,4.349657,4.310837,4.287938


In [21]:
# Write the output volatility to a csv file
df_volatility.to_csv('volatility.txt', sep='\t', encoding='utf-8', index = False)

In [23]:
df_volatility['stock'].unique().shape[0]

499